# Final Project

In [ ]:
import pandas as pd

In [ ]:
temperatures = pd.read_csv("data/temperature.csv")
temperatures['Date'] = pd.to_datetime(temperatures['Date'].astype(str), format='%Y%m').dt.to_period('M')

wildfires = pd.read_csv("data/wildfire.csv")
wildfires['Date'] = pd.to_datetime(wildfires['Date'].astype(str), format='%Y%m').dt.to_period('M')

hurricane = pd.read_csv("data/hurricane.csv")
hurricane['Date'] = hurricane['year'].astype(str) + "-" + hurricane['month'].astype(str).str.zfill(2) + "-" + hurricane['day'].astype(str).str.zfill(2)
hurricane['Date'] = pd.to_datetime(hurricane['Date'], format="%Y-%m-%d").dt.to_period('M')

tornadoes = pd.read_csv("data/tornado.csv")
tornadoes['Date'] = pd.to_datetime(tornadoes['Date'].astype(str), format='%Y%m').dt.to_period('M')

new_tornadoes = pd.read_csv("data/1950-2023_actual_tornadoes.csv")
new_tornadoes['Date'] = pd.to_datetime(new_tornadoes['date'], format="%Y-%m-%d").dt.to_period('M')

earthquake = pd.read_csv("data/eqint_tsqp.csv")
earthquake_usa = earthquake[(earthquake.COUNTRY == "USA") & (earthquake.LONGITUDE < 0)]

display(temperatures)
display(wildfires)
display(hurricane)
display(tornadoes)
display(new_tornadoes)

In [ ]:
from functools import reduce


super_dfs = [temperatures, tornadoes, wildfires, hurricane]
result = reduce(lambda left, right: pd.merge(left, right, on='Date', how='inner'), super_dfs)
display(result)

In [ ]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
from geodatasets import get_path

In [ ]:
gdf = geopandas.GeoDataFrame(
    earthquake_usa, geometry=geopandas.points_from_xy(earthquake_usa.LONGITUDE, earthquake_usa.LATITUDE), crs="EPSG:4326"
)
print(gdf.head())

In [ ]:
world = geopandas.read_file(get_path("naturalearth.land"))

# We restrict to South America.
ax = world.clip([-200, 10, -50, 72]).plot(color="white", edgecolor="black")

# We can now plot our ``GeoDataFrame``.
gdf.plot(ax=ax, color="red", markersize=1)

plt.show()

In [ ]:
# This example uses heatmaps to visualize the density of volcanoes
# which is more in some parts of the world compared to others.
import folium
from folium import plugins

map = folium.Map(location=[15, 30], tiles="Cartodb dark_matter", zoom_start=2)

heat_data = [[point.xy[1][0], point.xy[0][0]] for point in gdf.geometry]

heat_data
plugins.HeatMap(heat_data).add_to(map)

map